# Transfer Learning MNIST

* Train a simple convnet on the MNIST dataset the first 5 digits [0..4].
* Freeze convolutional layers and fine-tune dense layers for the classification of digits [5..9].

### Import MNIST data and create 2 datasets with one dataset having digits from 0 to 4 and other from 5 to 9 

#### Import the mnist dataset from keras datasets

In [1]:
import keras
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


#### Creating two datasets one with digits below 5 and one with 5 and above

In [0]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test >= 5] - 5

### Check 

Verify shapes of x_train, y_train, x_test and y_test for both the datasets with the below given shapes.

In [3]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [4]:
print(x_train_lt5.shape)
print(y_train_lt5.shape)
print(x_test_lt5.shape)
print(y_test_lt5.shape)

(30596, 28, 28)
(30596,)
(5139, 28, 28)
(5139,)


In [5]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28)
(29404,)
(4861, 28, 28)
(4861,)


### Let us take only the dataset (x_train, y_train, x_test, y_test) for Integers 0 to 4 in MNIST
### Reshape x_train and x_test to a 4 Dimensional array (channel = 1) to pass it into a Conv2D layer

In [0]:
x_train_lt5 = x_train_lt5.reshape(x_train_lt5.shape[0], 28, 28, 1)
x_test_lt5 = x_test_lt5.reshape(x_test_lt5.shape[0], 28, 28, 1)

### Change into float32 datatype and Normalize x_train and x_test by dividing it by 255.0

In [0]:
x_train_lt5 = x_train_lt5.astype('float32')
x_test_lt5 = x_test_lt5.astype('float32')

x_train_lt5/=255
x_test_lt5/=255

### Check

Verify the shapes of the X_train and X_test with the shapes given below.

In [8]:
print('X_train shape:', x_train_lt5.shape)
print('X_test shape:', x_test_lt5.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


In [9]:
print('X_train shape:', x_train_lt5.shape)
print('X_test shape:', x_test_lt5.shape)

X_train shape: (30596, 28, 28, 1)
X_test shape: (5139, 28, 28, 1)


### Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
from keras.utils import np_utils
number_of_classes = 5

Y_train = np_utils.to_categorical(y_train_lt5, number_of_classes)
Y_test = np_utils.to_categorical(y_test_lt5, number_of_classes)

### Build a sequential model with 2 Convolutional layers with 32 kernels of size (3,3) followed by a Max pooling layer of size (2,2) followed by a drop out layer to be trained for classification of digits 0-4  

In [11]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.optimizers import Adam
# input image dimensions
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)


#Initialize the model
model = Sequential()

#Add a Convolutional Layer with 32 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape,name='conv_1'))

#Add a Convolutional Layer with 64 filters of size 3X3 and activation function as 'ReLU' 
model.add(Conv2D(32, (3, 3), activation='relu',name='conv_2'))

#Add a MaxPooling Layer of size 2X2 
model.add(MaxPooling2D(pool_size=(2, 2),name='max_1'))

#Apply Dropout with 0.25 probability 
model.add(Dropout(0.25,name='drop_1'))



W0707 06:26:54.208148 140290285385600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0707 06:26:54.248738 140290285385600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0707 06:26:54.255506 140290285385600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0707 06:26:54.294544 140290285385600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0707 06:26:54.299688 140290285385600 deprecation_wrapp

### Post that flatten the data and add 2 Dense layers with 128 neurons and neurons = output classes with activation = 'relu' and 'softmax' respectively. Add dropout layer inbetween if necessary  

In [0]:
batch_size = 128
num_classes = 5
#epochs = 12
#Flatten the layer
model.add(Flatten())

#Add Fully Connected Layer with 128 units and activation function as 'ReLU'
model.add(Dense(128, activation='relu',name='dense_1'))

#Apply Dropout with 0.5 probability 
model.add(Dropout(0.5,name='drop_2'))

#Add Fully Connected Layer with 10 units and activation function as 'softmax'
model.add(Dense(num_classes, activation='softmax',name='dense_2'))

### Print the training and test accuracy for 5 epochs

In [13]:

from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

#To use adam optimizer for learning weights with learning rate = 0.001
optimizer = Adam(lr=0.001)
#Set the loss function and optimizer for the model training
model.compile(loss=categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])

W0707 06:27:02.169438 140290285385600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0707 06:27:02.178146 140290285385600 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [14]:
#Training on the dataset
model.fit(x_train_lt5, Y_train,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(x_test_lt5, Y_test))

W0707 06:27:05.055490 140290285385600 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 9s 302us/step - loss: 0.1326 - acc: 0.9589 - val_loss: 0.0186 - val_acc: 0.9938
Epoch 2/5
30596/30596 [==============================] - 2s 58us/step - loss: 0.0356 - acc: 0.9891 - val_loss: 0.0094 - val_acc: 0.9967
Epoch 3/5
30596/30596 [==============================] - 2s 58us/step - loss: 0.0239 - acc: 0.9935 - val_loss: 0.0066 - val_acc: 0.9975
Epoch 4/5
30596/30596 [==============================] - 2s 58us/step - loss: 0.0203 - acc: 0.9940 - val_loss: 0.0101 - val_acc: 0.9959
Epoch 5/5
30596/30596 [==============================] - 2s 59us/step - loss: 0.0164 - acc: 0.9951 - val_loss: 0.0056 - val_acc: 0.9981


### Use the model trained on 0 to 4 digit classification and train it on the dataset which has digits 5 to 9  (Using Transfer learning keeping only the dense layers to be trainable)

### Make only the dense layers to be trainable and convolutional layers to be non-trainable

In [15]:
x_train_gt5 = x_train_gt5.reshape(x_train_gt5.shape[0], 28, 28, 1)
x_test_gt5 = x_test_gt5.reshape(x_test_gt5.shape[0], 28, 28, 1)

x_train_gt5 = x_train_gt5.astype('float32')
x_test_gt5 = x_test_gt5.astype('float32')

x_train_gt5/=255
x_test_gt5/=255

print('X_train shape:', x_train_gt5.shape)
print('X_test shape:', x_test_gt5.shape)

from keras.utils import np_utils
number_of_classes = 5

Y_train_gt5 = np_utils.to_categorical(y_train_gt5, number_of_classes)
Y_test_gt5 = np_utils.to_categorical(y_test_gt5, number_of_classes)

X_train shape: (29404, 28, 28, 1)
X_test shape: (4861, 28, 28, 1)


#### Check model summary to see model layer names

In [16]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv_1
conv_1is not trainable

conv_2
conv_2is not trainable

max_1
max_1is not trainable

drop_1
drop_1is not trainable

flatten_1
flatten_1is not trainable

dense_1
dense_1 is trainable

drop_2
drop_2is not trainable

dense_2
dense_2 is trainable



In [17]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

conv_1
conv_1is not trainable

conv_2
conv_2is not trainable

max_1
max_1is not trainable

drop_1
drop_1is not trainable

flatten_1
flatten_1is not trainable

dense_1
dense_1 is trainable

drop_2
drop_2is not trainable

dense_2
dense_2 is trainable



### Do the required preprocessing for `x_train_gt5` also same as `x_train_lt5` and for `y_train_gt5` same as `y_train_lt5`

1. Reshape
2. Change to float32 datatype
3. Normalize (dividing with 255)
4. y_train and y_test Convert into one-hot vectors

In [0]:
#done above 

### Check

Verify the shapes with the given below.

In [18]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404,)
(4861, 28, 28, 1)
(4861,)


In [0]:
print(x_train_gt5.shape)
print(y_train_gt5.shape)
print(x_test_gt5.shape)
print(y_test_gt5.shape)

(29404, 28, 28, 1)
(29404, 5)
(4861, 28, 28, 1)
(4861, 5)


## Print the accuracy for classification of digits 5 to 9

In [19]:
#Training on the dataset
model.fit(x_train_gt5, Y_train_gt5,
          batch_size=batch_size,
          epochs=5,
          verbose=1,
          validation_data=(x_test_gt5, Y_test_gt5))

Train on 29404 samples, validate on 4861 samples
Epoch 1/5
 2304/29404 [=>............................] - ETA: 2s - loss: 1.3968 - acc: 0.6567

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


29404/29404 [==============================] - 2s 62us/step - loss: 0.2333 - acc: 0.9336 - val_loss: 0.0416 - val_acc: 0.9854
Epoch 2/5
29404/29404 [==============================] - 2s 59us/step - loss: 0.0662 - acc: 0.9804 - val_loss: 0.0280 - val_acc: 0.9897
Epoch 3/5
29404/29404 [==============================] - 2s 58us/step - loss: 0.0464 - acc: 0.9866 - val_loss: 0.0241 - val_acc: 0.9920
Epoch 4/5
29404/29404 [==============================] - 2s 59us/step - loss: 0.0377 - acc: 0.9880 - val_loss: 0.0232 - val_acc: 0.9926
Epoch 5/5
29404/29404 [==============================] - 2s 59us/step - loss: 0.0309 - acc: 0.9906 - val_loss: 0.0220 - val_acc: 0.9928


# Text classification using TF-IDF

###  Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### Training data

In [3]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

In [0]:
#twenty_train

### Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [0]:
#twenty_test

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [5]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [6]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [7]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# TfidfVectorizer
vect = TfidfVectorizer()
#pd.DataFrame(vect.fit_transform(twenty_train).toarray(), columns=vect.get_feature_names())
train_data=(vect.fit_transform(twenty_train.data))

In [0]:
#vect.vocabulary_

In [0]:
# TfidfVectorizer
#vect = TfidfVectorizer()
test_feature = (vect.transform(twenty_test.data))

### Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

In [12]:
logreg.fit(train_data, twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_pred = logreg.predict(test_feature)

In [14]:
train_data.shape

(2257, 35788)

In [16]:
from sklearn import metrics
metrics.accuracy_score(twenty_test.target, y_pred)

0.8868175765645806

In [0]:
y_train_pred = logreg.predict(train_data)

In [19]:
metrics.accuracy_score(twenty_train.target, y_train_pred)

0.9827204253433761